In [11]:
import pandas as pd

df = pd.read_csv('MODIS_C6_Australia_NewZealand_7d.csv')

display(df.head())

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight
0,-31.659,147.260,308.5,3.4,1.7,2020-05-15,100,T,67,6.0NRT,289.0,46.8,D
1,-31.652,147.254,312.2,3.4,1.7,2020-05-15,100,T,72,6.0NRT,289.1,67.4,D
2,-32.998,137.586,304.7,1.0,1.0,2020-05-15,100,T,32,6.0NRT,293.1,3.9,D
3,-33.007,137.585,305.8,1.0,1.0,2020-05-15,100,T,47,6.0NRT,292.9,4.8,D
4,-33.465,138.540,309.9,1.1,1.0,2020-05-15,100,T,60,6.0NRT,287.0,10.9,D


In [12]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import PRGn, RdYlGn
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter

import numpy as np


# Define function to switch from lat/long to mercator coordinates
def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

# Define coord as tuple (lat,long)
df['coordinates'] = list(zip(df['latitude'], df['longitude']))


# Obtain list of mercator coordinates
mercators = [x_coord(x, y) for x, y in df['coordinates'] ]


In [13]:

# Create mercator column in our df
df['mercator'] = mercators

# Split that column out into two separate cols - mercator_x and mercator_y
df[['mercator_x', 'mercator_y']] = df['mercator'].apply(pd.Series)

# Drop 'geometry' column 
df = df.drop(columns=['daynight', 'satellite', 'acq_time', 'version'])

# Examine our modified DataFrame
df.head()



,latitude,longitude,brightness,scan,track,acq_date,confidence,bright_t31,frp,coordinates,mercator,mercator_x,mercator_y
0,-31.659,147.260,308.5,3.4,1.7,2020-05-15,67,289.0,46.8,"(-31.659000000000002, 147.26)","(16392908.214217465, -3718631.8424891047)",1.639291e+07,-3.718632e+06
1,-31.652,147.254,312.2,3.4,1.7,2020-05-15,72,289.1,67.4,"(-31.651999999999997, 147.254)","(16392240.297272705, -3717716.406552673)",1.639224e+07,-3.717716e+06
2,-32.998,137.586,304.7,1.0,1.0,2020-05-15,32,293.1,3.9,"(-32.998000000000005, 137.586)","(15316003.46028334, -3895038.4998534755)",1.531600e+07,-3.895038e+06
3,-33.007,137.585,305.8,1.0,1.0,2020-05-15,47,292.9,4.8,"(-33.007, 137.585)","(15315892.140792545, -3896233.1331791007)",1.531589e+07,-3.896233e+06
4,-33.465,138.540,309.9,1.1,1.0,2020-05-15,60,287.0,10.9,"(-33.465, 138.54)","(15422202.254500119, -3957188.8405362014)",1.542220e+07,-3.957189e+06


In [20]:
df['acq_date'] = pd.to_datetime(df['acq_date']) - pd.to_timedelta(7, unit='d')
def wmy(df):
    return ([g for n, g in df.set_index('acq_date').groupby(pd.Grouper(freq = 'W'))],
            [g for n, g in df.set_index('acq_date').groupby(pd.Grouper(freq = 'M'))],
            [g for n, g in df.set_index('acq_date').groupby(pd.Grouper(freq = 'Y'))])
weeks, months, years = wmy(df)

In [21]:
# Create map with Bokeh

# Select tile set to use
chosentile = get_provider(Vendors.STAMEN_TONER)

# Choose palette
palette = PRGn[11]

# Tell Bokeh to use df as the source of the data
source = ColumnDataSource(data=weeks[0])

# Define color mapper - which column will define the colour of the data points
color_mapper = linear_cmap(field_name = 'brightness', palette = palette, low = weeks[0]['brightness'].min(), high = weeks[0]['brightness'].max())

# Set tooltips - these appear when we hover over a data point in our map, very nifty and very useful
nan_color = '#d9d9d9'
tooltips = [("Brightness","@brightness"), ("frp","@frp")]


# Create figure
p = figure(title = 'Australia fires', x_axis_type="mercator", y_axis_type="mercator", 
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

# Add map tile
p.add_tile(chosentile)

# Add points using mercator coordinates
p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=30, fill_alpha = 0.7)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

# Set color_bar location
p.add_layout(color_bar, 'right')

# Display in notebook
output_notebook()

# Save as HTML
output_file('australia.html', title='australia fires')

# Show map
show(p)



Loading BokehJS ...